In [1]:
#!/usr/bin/env python
from atproto import Client
import re
import peewee
from typing import Tuple
from datetime import datetime, timezone

# Database configuration
db = peewee.SqliteDatabase('feed_database2.db', pragmas={
    'journal_mode': 'wal',
    'cache_size': -1024 * 64
})

class BaseModel(peewee.Model):
    class Meta:
        database = db

class Post(BaseModel):
    uri = peewee.CharField(index=True)
    cid = peewee.CharField()
    reply_parent = peewee.CharField(null=True, default=None)
    reply_root = peewee.CharField(null=True, default=None)
    indexed_at = peewee.DateTimeField(default=datetime.utcnow)

def convert_web_url_to_at_uri(web_url: str) -> str:
    pattern = r"https://bsky\.app/profile/([^/]+)/post/([^/]+)"
    match = re.match(pattern, web_url)
    if match:
        handle, post_id = match.groups()
        return f"at://{handle}/app.bsky.feed.post/{post_id}"
    return None

def preview_post_details(client: Client, web_url: str) -> Tuple[str, str, str, str, datetime]:
    at_uri = convert_web_url_to_at_uri(web_url)
    if not at_uri:
        raise ValueError("Invalid Bluesky URL format")
    
    response = client.get_post_thread(uri=at_uri)
    post = response.thread.post
    
    # Get DID-based URI using author's DID
    uri = f"at://{post.author.did}/app.bsky.feed.post/{at_uri.split('/')[-1]}"
    
    reply_root = None
    reply_parent = None
    if hasattr(post.record, 'reply'):
        reply_root = post.record.reply.root.uri if post.record.reply !=None else None
        reply_parent = post.record.reply.parent.uri if post.record.reply !=None else None
    
    # Convert timestamp to UTC
    # Convert timestamp to naive datetime
    created_at = datetime.fromisoformat(post.record.created_at.replace('Z', '+00:00'))
    
    if created_at.tzinfo is not None:
        created_at = created_at.replace(tzinfo=None)  # Convert to naive datetime
    else:
        created_at = created_at.astimezone(timezone.utc)
    
    # Preview the values
    print("Preview of post details:")
    print(f"URI: {uri}")
    print(f"CID: {post.cid}")
    print(f"Created At (UTC): {created_at}")
    print(f"Reply Parent: {reply_parent}")
    print(f"Reply Root: {reply_root}")
    
    return uri, post.cid, reply_parent, reply_root, created_at
    

def add_post_to_db(details) -> None:
    try:
        uri, cid, reply_parent, reply_root, created_at = details
        
        with db.connection_context():
            existing_post = Post.select().where(Post.uri == uri).first()
            if existing_post:
                print("Post already exists in database")
                return
                
            Post.create(
                uri=uri,
                cid=cid,
                reply_parent=reply_parent,
                reply_root=reply_root,
                indexed_at=created_at  # Use original timestamp
            )
            print("Post successfully added to database")
            
    except Exception as e:
        print(f"Error processing post: {str(e)}")
    finally:
        if not db.is_closed():
            db.close()

In [2]:
usrnme = 'ramith.fyi'
psswd = '3mpu-vaed-7uvs-c4so'

client = Client()
client.login(usrnme, psswd);

In [5]:
# Preview the post details
web_url = "https://bsky.app/profile/amlab.bsky.social/post/3lc3pbropas2r"
details = preview_post_details(client, web_url)

Preview of post details:
URI: at://did:plc:6iowmsphpecdwmdzljt4jdqs/app.bsky.feed.post/3lc3pbropas2r
CID: bafyreih5djzdzkm7egxvv45rjndvemcrbua36mwaxzbfqjcevr2n2vahja
Created At (UTC): 2024-11-29 13:38:13.318000
Reply Parent: None
Reply Root: None


In [6]:
# Add to database only if preview looks correct
add_post_to_db(details)

Post successfully added to database
